In [3]:
import pandas as pd
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textblob import TextBlob
import string

In [13]:
data_file = open("data/yelp_academic_dataset_review.json")
reviews_data = []
c = 0
for line in data_file:
    reviews_data.append(json.loads(line))
    c+=1
    if c == 100000:
        break

reviews_df = pd.DataFrame(reviews_data)

data_file.close()

In [14]:
data_file = open("data/yelp_academic_dataset_business.json")
business_data = []

for line in data_file:
    business_data.append(json.loads(line))

business_df = pd.DataFrame(business_data)

data_file.close()

In [15]:
reviews_df.sort_values('date')

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
6942,TYslH-CAecjJxLNs96KduA,XCsZ3hWa_6oP1WkWvK7pmg,29YqJwOGEuAWqlHZxMc1OA,5.0,0,0,0,Lovely little restaurant which was converted f...,2005-03-01 17:47:15
5943,qU51pK9Ui3pG5phxzUKtyA,58yhbFfNHjULDZx0FD-Dvw,xwKYBPO0ByGlkvNcr8FdqQ,4.0,0,0,0,Romantic and cozy French restaurant. A bit pri...,2005-03-09 06:37:47
5471,bpraoKKp9AbNYbm1M6113A,58yhbFfNHjULDZx0FD-Dvw,B10mqANgHvL8gBteo1UhAA,5.0,0,0,0,My favorite Italian restaurant in Tucson. Fami...,2005-03-09 07:23:26
2756,QTE44bHheG6ugQU4em4SdQ,B6FbaEEn5Uh4kEqv4kUdgA,f82dhKNiUXsDVPMLqKYiIQ,3.0,0,0,0,"OK lunch buffet, but too oily. i think weeken...",2005-03-12 03:47:06
4689,ocPjTgIqZJaCnVehOaPmEQ,9Z2UEd_f6eC17YxifpWfMw,7n-8nhFq266slKmHJEY1-Q,3.0,1,0,0,"ya wanna get crunked? No problem, buy a 16oz o...",2005-03-14 23:43:07
...,...,...,...,...,...,...,...,...,...
5849,N0SEpi3lSDOMmNdjctk87Q,fCDCgLpxnflyE5LlFtSkEA,ErpOHBaOcmZjnBw0EutQJg,5.0,0,0,0,They have the best bagels in town! I have been...,2018-10-04 17:53:48
3011,hG0cjaWBeZ2sUKRHr5HSyA,WqbcV5y-yz1SNfdLZEYrvA,aKOdfQcjDG2NQtIEmq21ew,5.0,0,0,0,awesome place!!!! very dope setting large part...,2018-10-04 18:03:09
2423,v-K341pe1rxU_FtM_ubksQ,YIEafQT5RbTFbG8SnyO4LQ,fpQEKdtFprof-RJx1MRR9g,1.0,0,0,0,Locations at both Ina Rd and Oracle Rd have eq...,2018-10-04 18:10:01
8933,o0Jm0ALfZJp2JaX7sBv0Yg,jheopFKRPEDAR_ur5HGTug,pSmOH4a3HNNpYM82J5ycLA,3.0,0,0,0,Went with a large group and the service was de...,2018-10-04 18:20:00


In [16]:
# Filter out reviews for restaurants not in PA 
business_list = business_df[business_df['state']=='PA']['business_id']
df = reviews_df[reviews_df['business_id'].isin(business_list)]

In [22]:
def split_time_to_feature(df):

    df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
    
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['month'] = pd.DatetimeIndex(df['date']).month
    df['day'] = pd.DatetimeIndex(df['date']).day
    df['hour'] = pd.DatetimeIndex(df['date']).hour
    df['min'] = pd.DatetimeIndex(df['date']).minute
    df['sec'] = pd.DatetimeIndex(df['date']).second

split_time_to_feature(df)

/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_94901/789486673.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_94901/789486673.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = pd.DatetimeIndex(df['date']).year
/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_94901/789486673.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [23]:
# Extract reivews left in 2000-2018
df = df.loc[(df['year'] >= 2000) & (df['year'] <= 2018)]

### Feature engineering

In [24]:
# Creating a new column in the dataset for the length of the reviews
df['length_of_reviews'] = df['text'].apply(len)

In [25]:
# Creating a new column in the dataset for the number of words in the reviews
df['num_of_words'] = df['text'].apply(lambda str:len(nltk.word_tokenize(str)))

In [26]:
# Creating a new column in the dataset for the number of sentences in the reviews
df['num_of_sentences'] = df['text'].apply(lambda paragraph:len(nltk.sent_tokenize(paragraph)))

In [27]:
df['capital_words'] = df['text'].apply(lambda sen:len(re.findall(r'\b[A-Z]+\b', sen)))
df['capital_words_ratio'] = df['capital_words']/df['num_of_words']
df.drop(columns='capital_words', inplace = True)

### Dealing with texts in reviews

In [28]:
# Cleaning the reviews - stemed, remove stopwords and punctuation
def clean_text(text):

    ## Remove puncuation
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    ## Remove stop words
    nostop = [word for word in nopunc.split() if word.lower() not in stopwords.words('english') and len(word) >= 3]
    text = ' '.join(nostop)
   
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)    
    
    ps = PorterStemmer()
    text = ps.stem(text)

    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text)

    return text

df['cleaned_text'] = df['text'].apply(clean_text)

In [31]:
# Sentiment analysis using textblob 
df['sentiment_polarity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).polarity)
df['sentiment_subjectivity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).subjectivity)

In [33]:
df.to_csv('data/cleaned_review_data.csv')

In [35]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year,...,hour,min,sec,length_of_reviews,num_of_words,num_of_sentences,capital_words_ratio,cleaned_text,sentiment_polarity,sentiment_subjectivity
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,2018,...,22,9,11,513,114,7,0.026316,decide eat aware going take hours beginning en...,0.209722,0.419444
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,2012,...,15,28,18,829,174,7,0.011494,ive taken lot spin classes years nothing compa...,0.395455,0.571212
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,2015,...,0,1,3,243,56,6,0.000000,wow yummy different delicious favorite lamb cu...,0.279545,0.713068
5,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31,2015,...,23,10,31,341,79,10,0.063291,long term frequent customer establishment went...,0.211111,0.405556
7,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2,0,0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,2015,...,2,29,16,192,34,4,0.029412,amazingly amazing wings homemade bleu cheese r...,0.505556,0.788889
